In [1]:
class ResultObject:
    def __init__(self,final_annotations,initial_scores,corelation_matrix,common_de_genes):
        """Contructor function for SinglePython class.
    
        Parameters
        ----------
        final_annotations : DataFrame
            Final cell type annotations for each cell.
        
        initial_scores : DataFrame
            Initial scores of cell types for each cell.
            
        corelation_matrix : DataFrame
            Correlation matrix of cell types.
            
        common_de_genes : List
            Differentially highly expressed genes between each pair of cell types.
            
        """
        
        self.final_annotations=final_annotations
        self.initial_scores=initial_scores
        self.corelation_matrix=corelation_matrix
        self.common_de_genes=common_de_genes

In [11]:
import Result
import utils
import tuning
import scipy
import numpy as np
import pandas as pd

class SinglePythonObject:
    def __init__(self, scData, refDataset, annot=None, fine_tuning=False, tuning_by="top_n", tuning_threshold=0.05, tuning_top_n=7, min_gene_th=500, main_types = True):
        """Contructor function for SinglePython class.
    
        Parameters
        ----------
        sc_location : Str
            the location of sc-RNAseq data.
            
        refDataset : DataFrame
            The reference dataset gene expression matrix.
            
        annot : DataFrame
            Annotations for each column in ref_data.
            
        fine_tuning : Boolean
            If fine tuning will be applied.
            
        tuning_by : Str
            There are two tuning options. It can be ither done by taking top n most correlated cell types after
            the initial annotation (top_n) or setting a minimum threshold value to select cell types for tuning 
            (threshod). The default value is top_n.
            
        tuning_threshold : Float
            The threshold value for the selection of cell types for "threshold" tuning. Default value is 0.05.
            
        tuning_top_n : Int
            The number of top correlated cell types to be selected for "top_n" tuning. Default value is 7. 
            
        min_gene_th : Int
            Minimum gene threshold for genes that have expression more than zero. It is necessary to determine
            which cells to contribute to the analysis.
            
        de_genes_n : Int
            The number of top differentially expressed genes to be chosen for the analysis.
            
        """
        
        self.sc_data = utils.convertAnnDataToDf(scData)
        self.refDataset = refDataset.astype(float)
        if main_types == True:
            self.annot = pd.DataFrame(annot["level1"].values,columns = ["cellType"],index = annot.index)
        else:
            self.annot = pd.DataFrame(annot["level2"].values,columns = ["cellType"],index = annot.index)
        self.fine_tuning=fine_tuning
        self.tuning_by=tuning_by
        self.tuning_top_n=tuning_top_n
        self.tuning_threshold = tuning_threshold
        self.main_types = main_types
    def annotateCellTypes(self):
        """ Finds best annotation for single cells and Returns a Result object.
    
        """
        intersect=np.intersect1d(self.refDataset.index.values,self.sc_data.index)
        sc_data=self.sc_data.loc[intersect]
        refDataset=self.refDataset.loc[intersect]
        
        n=int(500*np.power(2/3,np.log2(len(np.unique(self.annot.cellType)))))
        
        de=utils.getDEgenes(refDataset,self.annot)
        de_merged=[]
        [de_merged.extend(i[:n]) for i in  de.values()]
        de_merged=np.unique(de_merged)
        print( len(de_merged))
        cor=scipy.stats.spearmanr(sc_data.loc[de_merged],refDataset.loc[de_merged])
        cor=pd.DataFrame(cor[0]).iloc[:,0:len(sc_data.columns)][-len(refDataset.columns):]
        cor.columns=sc_data.columns
        cor.index=refDataset.columns
        cor["cellType"]=self.annot["cellType"].values
        scores=cor.groupby("cellType").quantile(q=0.8)
        if self.fine_tuning==True:
            if (self.tuning_by=="top_n"):
                final_annotations=tuning._FineTuneByN(sc_data,refDataset,self.annot,de,scores,self.tuning_top_n)
                return Result.ResultObject(final_annotations,scores,cor,de_merged)
            elif (self.tuning_by=="threshold"):
                final_annotations=_FineTuneByT(sc_data,refDataset,self.annot,de,scores,self.tuning_threshold)
                return Result.ResultObject(final_annotations,scores,cor,de_merged)
            else:
                print("Undefined tuning method.")
        else:
            return Result.ResultObject(pd.DataFrame(scores.idxmax(),columns=["final_annotations"]),scores,cor,de)
  

In [16]:
import numpy as np
import pandas as pd
import scipy
import utils
import multiprocessing as mp

def _FineTuneByN(sc_data,refDataset,annot,de,scores,n):
    """ Applies fine tuning and return final annotations of single cells.
    Parameters
    ----------
    sc_data : DataFrame
        Sc-RNAseq data.
        
    refDataset : DataFrame
        The reference dataset gene expression matrix.
        
    annot : DataFrame
        Annotations for each column in ref_data.
        
    de : Dict
        Differentially expressed genes for each combination of cell types.
        
    scores : DataFrame
        Correlation scores from the first round.
        
    n : Int
        The number of top cell types to choose for the first fine tuning.
        
    Returns
    -------
    final_annotations : DataFrame
        A data frame with the final annotations of cell types for each single cell.
        
    """
    d={}
    [d.update({i:np.sort(scores.sort_index(by=i,ascending=False).index.values[0:n])}) for i in scores.columns]
    while(n>1):
        unique_types=[list(x) for x in set(tuple(x) for x in d.values())]
        for i in unique_types:
            cols=[j for j in d.keys() if list(d.get(j))==i]
            top_labels=i
            res=_FineTuneRoundByN(sc_data,refDataset,annot,top_labels,de,cols)
            [d.update({cols[t]:res[t]}) for t in range(len(cols))]
        n=n-1
    return pd.DataFrame(d,index=["final_annotations"])

def _FineTuneRoundByN(sc_data,refDataset,annot,top_labels,de,cols):
    """ Returns final annotations of single cells.
    Parameters
    ----------
    sc_data : DataFrame
        Sc-RNAseq data.
        
    refDataset : DataFrame
        The reference dataset gene expression matrix.
        
    annot : DataFrame
        Annotations for each column in ref_data.
        
    top_labels: List
        Most correlated cell types from the previous round.
        
    de : Dict
        Differentially expressed genes for each combination of cell types.
        
    cols : List
        Cell names to calculate correlations.
        
    Returns
    -------
    top_annotations : List
        A list of cell types that are associated with given cells.
        
    """
    annot=annot[annot["cellType"].isin(top_labels)]
    refDataset = refDataset.loc[:,annot.index]
    
    n=int(1000*np.power(2/3,np.log2(len(top_labels))))

    de_merged=[]
    [de_merged.extend(de.get(i)[:n]) for i in de.keys() if  i[0] in top_labels and i[1] in top_labels]
    de_merged=np.unique(de_merged)

    cor=scipy.stats.spearmanr(sc_data.loc[de_merged,cols],refDataset.loc[de_merged])
    cor=pd.DataFrame(cor[0]).iloc[:,0:len(cols)][-len(refDataset.columns):]
    cor.columns=cols
    cor.index=refDataset.columns
    cor["cellType"]=annot["cellType"].values
    scores=cor.groupby("cellType").quantile(q=0.8)
    return [scores.sort_index(by=i,ascending=False).index.values[0:len(top_labels)-1] for i in scores.columns]

def _FineTuneByT(sc_data,refDataset,annot,de,scores,threshold):
    """ Applies fine tuning and return final annotations of single cells.
    Parameters
    ----------
    sc_data : DataFrame
        Sc-RNAseq data.
        
    refDataset : DataFrame
        The reference dataset gene expression matrix.
        
    annot : DataFrame
        Annotations for each column in ref_data.
        
    de : Dict
        Differentially expressed genes for each combination of cell types.
        
    scores : DataFrame
        Correlation scores from the first round.
        
    threshold : Float
        The cutoff value for correlation scores of cell types to choose for the first fine tuning.
        
    Returns
    -------
    final_annotations : DataFrame
        A data frame with the final annotations of cell types for each single cell.
        
    """
    final_annotations=pd.DataFrame(index=["annotation"])
    for i in sc_data.columns:
        top_labels=scores[scores[i]>max(scores[i])-threshold].dropna().index.values
        while(len(top_labels)>1):
            top_labels=_FineTuneRoundByT(sc_data,refDataset,annot,top_labels,de,i,threshold)
        final_annotations[i]=top_labels[0]
    return final_annotations

def _FineTuneRoundByT(sc_data,refDataset,annot,top_labels,de,i,threshold):
    """ Returns final annotations of single cells.
    Parameters
    ----------
    sc_data : DataFrame
        Sc-RNAseq data.
        
    refDataset : DataFrame
        The reference dataset gene expression matrix.
        
    annot : DataFrame
        Annotations for each column in ref_data.
        
    top_labels: List
        Most correlated cell types from the previous round.
        
    de : Dict
        Differentially expressed genes for each combination of cell types.
        
    i : String
        The cell name to calculate correlations for.
        
    Returns
    -------
    top_annotations : List
        A list of cell types that are associated with given cells.
        
    """
    annot=annot[annot["cellType"].isin(top_labels)]
    refDataset = refDataset.loc[:,annot.index]

    n=int(1000*np.power(2/3,np.log2(len(top_labels))))
    print("N: "+ str(n))
    de_merged=[]
    [de_merged.extend(de.get(i)[:n]) for i in de.keys() if  i[0] in top_labels and i[1] in top_labels]
    de_merged=np.unique(de_merged)
    print("Labels: "+ str(top_labels))
    print("N of de genes: " + str(len(de_merged)))
    if len(de_merged) < 20:
        return top_labels[0]
    if np.std(sc_data.loc[de_merged,i]) > 0:
        cor=scipy.stats.spearmanr(sc_data.loc[de_merged,i],refDataset.loc[de_merged])
        cor=pd.DataFrame(cor[0]).iloc[:,0:1][-len(refDataset.columns):]
        cor.columns=[i]
        cor.index=refDataset.columns
        cor["cellType"]=annot["cellType"].values
        scores=cor.groupby("cellType").quantile(q=0.8)
        scores=scores.sort_values(by=i,ascending=False)
        scores=scores.drop(scores.index[-1])
        return scores[scores[i]>max(scores[i])-threshold].dropna().index.values
    else:
        return top_labels[0]

In [13]:
import numpy as np
import scanpy.api as sc
import pandas as pd
import os
import scipy
import seaborn as sns
from matplotlib import pyplot as plt

def getDEgenes(refDataset,annot,n=200):
    """ Creates a dictionary for differentially highly expressed genes for all pairwise cell types in the a reference data set.
    
    Parameters
    ----------
    refDataset : DataFrame
        The reference dataset gene expression matrix.
        
    annot : DataFrame
        Annotations for each column in ref_data.
        
    Returns
    -------
    deGenes : dict with multiple index
        Dictionary containing differentially highly expressed genes for each combination of cell types. 
    """
    types = annot.groupby("cellType").groups.keys()
    median = refDataset.groupby(annot.cellType.values,axis=1).median()
    medDiff = {}
    [medDiff.update({(i,j):median[i]-median[j]}) for i in types  for j in types if i!=j]
    deGenes={}
    [deGenes.update({i: median.index[medDiff.get(i).argsort()[0:200]]}) for i in medDiff.keys()]
    return deGenes

def readDataDGE(path):
    data = sc.read_text(path,
                delimiter = "\t", first_column_names = True).transpose()
    
    data.var_names_make_unique()
    
    sc.pp.filter_cells(data, min_genes=1) # Hack to generate the n_genes column
    sc.pp.filter_genes(data, min_cells=1)
    
    return data
    
def readData_SingleR(path,min_genes):
    """ Reads, precesses and returns single cell data as a matrix. 
    
    Parameters
    ----------
    path : Str
        Diractory path, the location of single cell data.
        
    min_genes : Int
        The minimum number of genes for a cell to have in order to participate the analysis.
        
    Returns
    -------
    sc_data : DataFrame
        Single cell data matrix. 
        
    """
    result = sc.read(path + 'matrix.mtx').transpose() #, cache=True
    result.var_names = np.genfromtxt(path + 'genes.tsv', dtype=str)[:, 1]
    result.obs_names = np.genfromtxt(path + 'barcodes.tsv', dtype=str)
    result.var_names_make_unique()
    result.obs['n_counts'] = np.sum(result.X, axis=1).A1
    sc.pp.filter_cells(result, min_genes=min_genes)
    
    return result

def readCountMartix(path,min_genes):
    """ 
    Reads, precesses and returns single cell data. 
    
    Parameters
    ----------
    path : Str
        Diractory path, the location of single cell data.
        
    min_genes : Int
        The minimum number of genes for a cell to have in order to participate the analysis.
        
    Returns
    -------
    scData : AnnData
        Single cell data. 
        
    """
    
    result = sc.read_csv(path).transpose() #, cache=True
    result.var_names_make_unique()
    n_counts = np.sum(result.X, axis=1)
    result.obs['n_counts'] = n_counts
    sc.pp.filter_cells(result, min_genes=min_genes)
    
    return result

def read10xData(path,min_genes):
    """ 
    Reads, precesses and returns single cell data. 
    
    Parameters
    ----------
    path : Str
        Diractory path, the location of single cell data.
        
    min_genes : Int
        The minimum number of genes for a cell to have in order to participate the analysis.
        
    Returns
    -------
    scData : AnnData
        Single cell data. 
        
    """
    
    result = sc.read(path + 'matrix.mtx').transpose() #, cache=True
    result.var_names = np.genfromtxt(path + 'genes.tsv', dtype=str)[:, 1]
    result.obs_names = np.genfromtxt(path + 'barcodes.tsv', dtype=str)
    result.var_names_make_unique()
    result.obs['n_counts'] = np.sum(result.X, axis=1).A1
    sc.pp.filter_cells(result, min_genes=min_genes)
    
    return result
def readSCData(path,min_genes):
    """ 
    Reads, precesses and returns single cell data. 
    
    Parameters
    ----------
    path : Str
        Diractory path, the location of single cell data.
        
    min_genes : Int
        The minimum number of genes for a cell to have in order to participate the analysis.
        
    Returns
    -------
    scData : AnnData
        Single cell data. 
        
    """
    
    if "matrix.mtx" in os.listdir(path):
        result = read10xData(path, min_genes)
        return result
    else:
        try:
            data_file = [i for i in os.listdir(path) if "expression" in i][0]
            result = readCountMartix(path+data_file, min_genes)
            return result
        except:
            print("Please end your single cell data count matrix file with word 'expression'.")




def convertAnnDataToDf(scData):    
    try:
        result = pd.DataFrame(scData.X.toarray()) # If data is 10x data
    except:
        result = pd.DataFrame(scData.X[:]) # If data is Digital Gene expression matrix
        
    result.index = scData.obs_names.values
    result.columns = scData.var_names.values
    return result.T

def majorCellTypeCount(annotationResult):
    
    def splitAndReturnFirst(x):
        return x.split(":")[0]

    majorCellTypes = list(set(map(splitAndReturnFirst, annotationResult.final_annotations.transpose()["final_annotations"].tolist())))
    majorCellTypes.sort()

    result = pd.DataFrame()

    for i in majorCellTypes:
        totalCells = len(annotationResult.final_annotations.columns)
        cellTypeCount = len(annotationResult.final_annotations.transpose()["final_annotations"][annotationResult.final_annotations.transpose()["final_annotations"].str.startswith(i)])

        result = result.append([{
            "cellType" : i,
            "cellTypeCount" : cellTypeCount,
            "percentageOfCellType" : cellTypeCount / totalCells
        }])


    return result.set_index("cellType").sort_values("cellTypeCount", ascending = False)


def _plotHistograms(initial_scores):
    f, ax = plt.subplots(1, 1)
    histMatrix(initial_scores)
    sns.distplot(initial_scores.max(),hist = False,label="Max per cell")
    sns.distplot(initial_scores.min(),hist = False,label="Min per cell")
    ax.legend()
    plt.title("Correlation distributions")
    plt.show()

def histMatrix(matrix):
    list_of_lists = matrix.values.tolist()
    list_of_fluxes = [item for sublist in list_of_lists for item in sublist]
    sns.set(style="white", palette="muted",color_codes=True)
    plot = sns.distplot(list_of_fluxes,hist = False,label="All")

In [5]:
refDataset = pd.read_csv("./Data/Reference/CHETAH/chetah_ref_data.csv", sep=";", decimal = ",", index_col = 0)
refDataset = refDataset.astype('float')

annot = pd.read_csv("./Data/Reference/CHETAH/chetah_ref_annot.csv",sep=";", index_col = 0)
annot.columns = ["level1"]

In [6]:
data =readSCData("C:/Users/murat_gga8ya6/Desktop/Thesis/Datasets/Smart-Seq2/Melanoma/",200)

Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.


In [14]:
obj = SinglePythonObject(refDataset=refDataset,annot=annot,scData=data,fine_tuning=True,tuning_by="threshold")

In [17]:
result = obj.annotateCellTypes()

1438
N: 525
Labels: ['CD4 T cell' 'CD8 T cell' 'reg. T cell']
N of de genes: 664
N: 525
Labels: ['CD4 T cell' 'CD8 T cell' 'reg. T cell']
N of de genes: 664
N: 525
Labels: ['CD4 T cell' 'CD8 T cell' 'reg. T cell']
N of de genes: 664
N: 666
Labels: ['CD8 T cell' 'CD4 T cell']
N of de genes: 400
N: 666
Labels: ['CD4 T cell' 'CD8 T cell']
N of de genes: 400
N: 444
Labels: ['B cell' 'CD8 T cell' 'Macrophage' 'reg. T cell']
N of de genes: 953
N: 666
Labels: ['CD8 T cell' 'reg. T cell']
N of de genes: 400
N: 525
Labels: ['CD4 T cell' 'CD8 T cell' 'reg. T cell']
N of de genes: 664
N: 666
Labels: ['CD8 T cell' 'CD4 T cell']
N of de genes: 400
N: 525
Labels: ['CD4 T cell' 'CD8 T cell' 'reg. T cell']
N of de genes: 664
N: 666
Labels: ['CD8 T cell' 'CD4 T cell']
N of de genes: 400
N: 525
Labels: ['CD4 T cell' 'CD8 T cell' 'reg. T cell']
N of de genes: 664
N: 666
Labels: ['CD4 T cell' 'reg. T cell']
N of de genes: 400
N: 525
Labels: ['CD4 T cell' 'CD8 T cell' 'reg. T cell']
N of de genes: 664
N: 6

N: 525
Labels: ['CD4 T cell' 'CD8 T cell' 'reg. T cell']
N of de genes: 664
N: 666
Labels: ['CD8 T cell' 'reg. T cell']
N of de genes: 400
N: 390
Labels: ['B cell' 'CD4 T cell' 'CD8 T cell' 'Fibroblast' 'reg. T cell']
N of de genes: 1060
N: 444
Labels: ['Fibroblast' 'reg. T cell' 'CD4 T cell' 'B cell']
N of de genes: 926
N: 525
Labels: ['CD4 T cell' 'reg. T cell' 'Fibroblast']
N of de genes: 702
N: 666
Labels: ['reg. T cell' 'Fibroblast']
N of de genes: 400
N: 390
Labels: ['B cell' 'CD4 T cell' 'CD8 T cell' 'Dendritic' 'reg. T cell']
N of de genes: 1075
N: 444
Labels: ['CD4 T cell' 'CD8 T cell' 'reg. T cell' 'Dendritic']
N of de genes: 893
N: 525
Labels: ['CD4 T cell' 'CD8 T cell' 'reg. T cell']
N of de genes: 664
N: 666
Labels: ['CD4 T cell' 'reg. T cell']
N of de genes: 400
N: 525
Labels: ['CD4 T cell' 'CD8 T cell' 'reg. T cell']
N of de genes: 664
N: 525
Labels: ['CD4 T cell' 'CD8 T cell' 'reg. T cell']
N of de genes: 664
N: 666
Labels: ['CD4 T cell' 'CD8 T cell']
N of de genes: 400

N: 390
Labels: ['B cell' 'CD8 T cell' 'Macrophage' 'Mast' 'reg. T cell']
N of de genes: 1093
N: 444
Labels: ['reg. T cell' 'CD8 T cell' 'B cell' 'Mast']
N of de genes: 835
N: 525
Labels: ['reg. T cell' 'CD8 T cell' 'B cell']
N of de genes: 675
N: 666
Labels: ['CD8 T cell' 'reg. T cell']
N of de genes: 400
N: 525
Labels: ['CD4 T cell' 'CD8 T cell' 'reg. T cell']
N of de genes: 664
N: 666
Labels: ['CD4 T cell' 'CD8 T cell']
N of de genes: 400
N: 525
Labels: ['CD4 T cell' 'CD8 T cell' 'reg. T cell']
N of de genes: 664
N: 525
Labels: ['CD4 T cell' 'CD8 T cell' 'reg. T cell']
N of de genes: 664
N: 666
Labels: ['CD8 T cell' 'reg. T cell']
N of de genes: 400
N: 525
Labels: ['CD4 T cell' 'CD8 T cell' 'reg. T cell']
N of de genes: 664
N: 666
Labels: ['CD4 T cell' 'CD8 T cell']
N of de genes: 400
N: 525
Labels: ['CD4 T cell' 'CD8 T cell' 'reg. T cell']
N of de genes: 664
N: 666
Labels: ['reg. T cell' 'CD8 T cell']
N of de genes: 400
N: 666
Labels: ['CD4 T cell' 'reg. T cell']
N of de genes: 400


N: 666
Labels: ['CD8 T cell' 'reg. T cell']
N of de genes: 400
N: 525
Labels: ['CD4 T cell' 'CD8 T cell' 'reg. T cell']
N of de genes: 664
N: 666
Labels: ['CD8 T cell' 'reg. T cell']
N of de genes: 400
N: 525
Labels: ['CD4 T cell' 'CD8 T cell' 'reg. T cell']
N of de genes: 664
N: 666
Labels: ['CD4 T cell' 'CD8 T cell']
N of de genes: 400
N: 525
Labels: ['CD4 T cell' 'CD8 T cell' 'reg. T cell']
N of de genes: 664
N: 666
Labels: ['CD8 T cell' 'reg. T cell']
N of de genes: 400
N: 525
Labels: ['CD4 T cell' 'CD8 T cell' 'reg. T cell']
N of de genes: 664
N: 666
Labels: ['CD8 T cell' 'CD4 T cell']
N of de genes: 400
N: 525
Labels: ['CD4 T cell' 'CD8 T cell' 'reg. T cell']
N of de genes: 664
N: 666
Labels: ['CD8 T cell' 'reg. T cell']
N of de genes: 400
N: 525
Labels: ['CD4 T cell' 'CD8 T cell' 'reg. T cell']
N of de genes: 664
N: 666
Labels: ['CD8 T cell' 'reg. T cell']
N of de genes: 400
N: 525
Labels: ['CD4 T cell' 'CD8 T cell' 'reg. T cell']
N of de genes: 664
N: 666
Labels: ['CD8 T cell' 

Labels: ['CD4 T cell' 'CD8 T cell' 'reg. T cell']
N of de genes: 664
N: 666
Labels: ['CD8 T cell' 'reg. T cell']
N of de genes: 400
N: 525
Labels: ['CD4 T cell' 'CD8 T cell' 'reg. T cell']
N of de genes: 664
N: 666
Labels: ['CD8 T cell' 'reg. T cell']
N of de genes: 400
N: 444
Labels: ['CD4 T cell' 'CD8 T cell' 'Dendritic' 'reg. T cell']
N of de genes: 893
N: 525
Labels: ['CD4 T cell' 'CD8 T cell' 'reg. T cell']
N of de genes: 664
N: 525
Labels: ['CD4 T cell' 'CD8 T cell' 'reg. T cell']
N of de genes: 664
N: 525
Labels: ['CD4 T cell' 'CD8 T cell' 'reg. T cell']
N of de genes: 664
N: 666
Labels: ['CD4 T cell' 'reg. T cell']
N of de genes: 400
N: 444
Labels: ['B cell' 'CD4 T cell' 'CD8 T cell' 'reg. T cell']
N of de genes: 888
N: 525
Labels: ['CD4 T cell' 'CD8 T cell' 'reg. T cell']
N of de genes: 664
N: 666
Labels: ['CD4 T cell' 'CD8 T cell']
N of de genes: 400
N: 525
Labels: ['CD4 T cell' 'CD8 T cell' 'reg. T cell']
N of de genes: 664
N: 666
Labels: ['CD8 T cell' 'reg. T cell']
N of de

...

N: 525
Labels: ['CD4 T cell' 'CD8 T cell' 'reg. T cell']
N of de genes: 664
N: 666
Labels: ['CD8 T cell' 'reg. T cell']
N of de genes: 400
N: 525
Labels: ['CD4 T cell' 'CD8 T cell' 'reg. T cell']
N of de genes: 664
N: 666
Labels: ['CD8 T cell' 'CD4 T cell']
N of de genes: 400
N: 525
Labels: ['CD4 T cell' 'CD8 T cell' 'reg. T cell']
N of de genes: 664
N: 525
Labels: ['CD4 T cell' 'CD8 T cell' 'reg. T cell']
N of de genes: 664
N: 666
Labels: ['CD4 T cell' 'CD8 T cell']
N of de genes: 400
N: 525
Labels: ['CD4 T cell' 'CD8 T cell' 'reg. T cell']
N of de genes: 664
N: 525
Labels: ['CD4 T cell' 'CD8 T cell' 'reg. T cell']
N of de genes: 664
N: 666
Labels: ['CD8 T cell' 'reg. T cell']
N of de genes: 400
N: 525
Labels: ['CD4 T cell' 'CD8 T cell' 'reg. T cell']
N of de genes: 664
N: 666
Labels: ['CD8 T cell' 'reg. T cell']
N of de genes: 400
N: 525
Labels: ['CD4 T cell' 'CD8 T cell' 'reg. T cell']
N of de genes: 664
N: 666
Labels: ['CD8 T cell' 'reg. T cell']
N of de genes: 400
N: 525
Labels: [

Labels: ['CD8 T cell' 'CD4 T cell']
N of de genes: 400
N: 525
Labels: ['CD4 T cell' 'CD8 T cell' 'reg. T cell']
N of de genes: 664
N: 666
Labels: ['CD8 T cell' 'reg. T cell']
N of de genes: 400
N: 525
Labels: ['CD4 T cell' 'CD8 T cell' 'reg. T cell']
N of de genes: 664
N: 666
Labels: ['CD8 T cell' 'CD4 T cell']
N of de genes: 400
N: 525
Labels: ['CD4 T cell' 'CD8 T cell' 'reg. T cell']
N of de genes: 664
N: 666
Labels: ['CD8 T cell' 'CD4 T cell']
N of de genes: 400
N: 666
Labels: ['CD8 T cell' 'reg. T cell']
N of de genes: 400
N: 525
Labels: ['CD4 T cell' 'CD8 T cell' 'reg. T cell']
N of de genes: 664
N: 666
Labels: ['CD4 T cell' 'CD8 T cell']
N of de genes: 400
N: 525
Labels: ['CD4 T cell' 'CD8 T cell' 'reg. T cell']
N of de genes: 664
N: 666
Labels: ['CD8 T cell' 'reg. T cell']
N of de genes: 400
N: 525
Labels: ['CD4 T cell' 'CD8 T cell' 'reg. T cell']
N of de genes: 664
N: 666
Labels: ['CD8 T cell' 'reg. T cell']
N of de genes: 400
N: 525
Labels: ['CD4 T cell' 'CD8 T cell' 'reg. T c

N: 525
Labels: ['CD4 T cell' 'CD8 T cell' 'reg. T cell']
N of de genes: 664
N: 666
Labels: ['CD8 T cell' 'CD4 T cell']
N of de genes: 400
N: 525
Labels: ['CD4 T cell' 'CD8 T cell' 'reg. T cell']
N of de genes: 664
N: 666
Labels: ['reg. T cell' 'CD8 T cell']
N of de genes: 400
N: 525
Labels: ['CD4 T cell' 'CD8 T cell' 'reg. T cell']
N of de genes: 664
N: 666
Labels: ['CD8 T cell' 'CD4 T cell']
N of de genes: 400
N: 525
Labels: ['CD4 T cell' 'CD8 T cell' 'reg. T cell']
N of de genes: 664
N: 666
Labels: ['CD8 T cell' 'reg. T cell']
N of de genes: 400
N: 525
Labels: ['CD4 T cell' 'CD8 T cell' 'reg. T cell']
N of de genes: 664
N: 666
Labels: ['CD8 T cell' 'CD4 T cell']
N of de genes: 400
N: 525
Labels: ['CD4 T cell' 'CD8 T cell' 'reg. T cell']
N of de genes: 664
N: 525
Labels: ['CD8 T cell' 'Macrophage' 'reg. T cell']
N of de genes: 705
N: 525
Labels: ['CD4 T cell' 'CD8 T cell' 'reg. T cell']
N of de genes: 664
N: 666
Labels: ['CD8 T cell' 'reg. T cell']
N of de genes: 400
N: 525
Labels: ['

N: 525
Labels: ['CD4 T cell' 'CD8 T cell' 'reg. T cell']
N of de genes: 664
N: 525
Labels: ['CD4 T cell' 'CD8 T cell' 'reg. T cell']
N of de genes: 664
N: 666
Labels: ['CD8 T cell' 'CD4 T cell']
N of de genes: 400
N: 525
Labels: ['CD4 T cell' 'CD8 T cell' 'reg. T cell']
N of de genes: 664
N: 666
Labels: ['CD4 T cell' 'CD8 T cell']
N of de genes: 400
N: 525
Labels: ['CD4 T cell' 'CD8 T cell' 'reg. T cell']
N of de genes: 664
N: 666
Labels: ['CD8 T cell' 'CD4 T cell']
N of de genes: 400
N: 525
Labels: ['CD4 T cell' 'CD8 T cell' 'reg. T cell']
N of de genes: 664
N: 666
Labels: ['CD8 T cell' 'reg. T cell']
N of de genes: 400
N: 666
Labels: ['CD8 T cell' 'reg. T cell']
N of de genes: 400
N: 525
Labels: ['CD4 T cell' 'CD8 T cell' 'reg. T cell']
N of de genes: 664
N: 666
Labels: ['CD4 T cell' 'CD8 T cell']
N of de genes: 400
N: 525
Labels: ['CD4 T cell' 'CD8 T cell' 'reg. T cell']
N of de genes: 664
N: 666
Labels: ['CD8 T cell' 'reg. T cell']
N of de genes: 400
N: 525
Labels: ['CD4 T cell' 'C

Labels: ['CD8 T cell' 'reg. T cell']
N of de genes: 400
N: 525
Labels: ['CD4 T cell' 'CD8 T cell' 'reg. T cell']
N of de genes: 664
N: 666
Labels: ['CD8 T cell' 'CD4 T cell']
N of de genes: 400
N: 525
Labels: ['CD4 T cell' 'CD8 T cell' 'reg. T cell']
N of de genes: 664
N: 525
Labels: ['CD4 T cell' 'CD8 T cell' 'reg. T cell']
N of de genes: 664
N: 666
Labels: ['CD8 T cell' 'CD4 T cell']
N of de genes: 400
N: 525
Labels: ['CD4 T cell' 'CD8 T cell' 'reg. T cell']
N of de genes: 664
N: 666
Labels: ['CD8 T cell' 'reg. T cell']
N of de genes: 400
